In [16]:
!pip install --upgrade pip
!pip install --quiet allensdk neuron

In [17]:
import math
import matplotlib.pyplot as plt
from allensdk.api.queries.biophysical_api import BiophysicalApi
from allensdk.model.biophys_sim.config import Config
from allensdk.model.biophysical.utils import Utils


In [18]:
specimen_id = input("Enter the specimen ID: ")
cell_id = input("Enter the cell ID to download from AllenDB: ")
desired_R_in = float(input("Enter the desired input resistance (in MOhm): "))

Enter the specimen ID: 488683425
Enter the cell ID to download from AllenDB: 491766131
Enter the desired input resistance (in MOhm): 0.2


In [19]:
bp = BiophysicalApi()
bp.cache_stimulus = False
print(f"Downloading cell data for cell ID {cell_id}...")
bp.cache_data(int(cell_id))
print("Download complete.")

HTTPError: HTTP Error 503: Service Unavailable

In [20]:
description = Config().load('manifest.json')
utils = Utils(description)
h = utils.h

ERROR:allensdk.config.app.application_config:Could not load configuration file: manifest.json
[Errno 2] No such file or directory: 'manifest.json'


FileNotFoundError: [Errno 2] No such file or directory: 'manifest.json'

In [ ]:
morphology_path = description.manifest.get_path('MORPHOLOGY')
utils.generate_morphology(morphology_path.encode('ascii', 'ignore'))

# Set default passive properties
passive = {
    "ra": 100,
    "cm": [
        {"section": "soma", "cm": 1},
        {"section": "axon", "cm": 1},
        {"section": "dend", "cm": 1},
        {"section": "apic", "cm": 1}
    ],
    "e_pas": -65.75
}
utils.description.data["passive"][0] = passive
utils.load_cell_parameters()

In [ ]:
soma_area = 0.0
for sec in h.soma:
    # Approximate surface area for a cylinder: area = π * diameter * length
    soma_area += math.pi * sec.diam * sec.L
print(f"Estimated soma area: {soma_area:.2f} µm²")

In [ ]:
new_gbar_leak = (1 / desired_R_in) / soma_area
print(f"Updating g-bar-leak to {new_gbar_leak:.5f} (units depend on model specifics)")
utils.description.data["passive"][0]["gbar_leak"] = new_gbar_leak
utils.load_cell_parameters()

In [ ]:
# Set up a negative current injection at the soma
stim = h.IClamp(h.soma[0](0.5))
stim.amp = -0.1
stim.delay = 100.0
stim.dur = 800.0
h.tstop = 1000.0

# Record voltage data using provided utility
vec = utils.record_values()

In [ ]:
# Initialize and run the simulation
h.finitialize()
h.run()

In [ ]:
# Convert recorded vectors to NumPy arrays for analysis
t = vec['t'].as_numpy()
v = vec['v'].as_numpy()

# Estimate resting voltage (average before current injection, t < 100 ms)
V_rest = v[t < 100].mean()

In [ ]:
# Find the minimum voltage during the injection period (100 ms <= t <= 900 ms)
injection_indices = (t >= 100) & (t <= 900)
V_trough = v[injection_indices].min()

# Compute the estimated input resistance
R_in_estimated = (V_rest - V_trough) / 0.1

In [ ]:
print(f"\nEstimated input resistance from simulation: {R_in_estimated:.2f} MOhm")

plt.figure(figsize=(8, 4))
plt.plot(t, v)
plt.xlabel("Time (ms)")
plt.ylabel("Voltage (mV)")
plt.title("Voltage Trace during Negative Current Injection")
plt.show()